# Download data

In [1]:
!wget https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Training_Images.zip

--2024-01-19 14:45:08--  https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Training_Images.zip
Resolving sid.erda.dk (sid.erda.dk)... 130.225.104.13
Connecting to sid.erda.dk (sid.erda.dk)|130.225.104.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 276294756 (263M) [application/zip]
Saving to: ‘GTSRB_Final_Training_Images.zip’

GTSRB_Final_Trainin 100%[===================>] 263.50M  20.3MB/s    in 15s     

2024-01-19 14:45:23 (17.8 MB/s) - ‘GTSRB_Final_Training_Images.zip’ saved [276294756/276294756]



In [2]:
!unzip /content/GTSRB_Final_Training_Images.zip

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
  inflating: GTSRB/Final_Training/Images/00035/00000_00019.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00020.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00021.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00022.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00023.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00024.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00025.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00026.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00027.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00028.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00029.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00001_00000.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00001_00001.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00001_00002.ppm  
  inflating: GTSRB/Final_Training

# Convert to YOLO format

In [3]:
import os
import pandas as pd
import shutil
import cv2

In [ ]:
os_dir ='/content/GTSRB/Final_Training/Images'
## DANGEROUS CODE --- BEWARE ###
for dir in os.listdir(os_dir):
    if dir == '.DS_Store' :
        continue
    inner_dir = os.path.join(os_dir, dir)
    # print(inner_dir[-5:])
    for img in os.listdir(inner_dir):
        if img == "GT-" + dir + '.csv':
            csv_file = pd.read_csv(os.path.join(inner_dir,"GT-" + dir + '.csv'), sep=';')
            csv_file['Filename'] = csv_file['Filename'].apply(lambda x: f'{inner_dir[-5:]}_{x}'.replace('ppm', 'jpg'))
            csv_file.to_csv(f'{inner_dir}/GGT-{dir}.csv', sep=';',index = False)
        else:
            continue
            os.rename(inner_dir + '/' + img, inner_dir + '/' + f'{inner_dir[-5:]}_{img}')

os.makedirs('YOLO/images', exist_ok=True)

for dir in os.listdir(os_dir):
    if dir == '.DS_Store':
        continue
    inner_dir = os.path.join(os_dir, dir)
    # print(inner_dir[-5:])
    for img in os.listdir(inner_dir):
        if img == "GT-" + dir + '.csv' or img == "GGT-" + dir + '.csv' or img == '00000_GT-00000.gsheet': # omit all files except the images
            continue
        else:
            image = cv2.imread(f'{inner_dir}/{img}')
            cv2.imwrite(f"YOLO/images/{inner_dir[-5:]}_{img.replace('ppm', 'jpg')}", image)

train_csv = []
for dir in os.listdir(os_dir):
    if dir == '.DS_Store' :
        continue
    inner_dir = os.path.join(os_dir, dir)
    csv_file = pd.read_csv(os.path.join(inner_dir,"GGT-" + dir + '.csv'), sep=';')

    train_csv.append(csv_file)

train_csv = pd.concat(train_csv, axis=0)
print(train_csv.shape)
train_csv = train_csv.reset_index().drop('index', axis=1)
train_csv

(39209, 8)


,Filename,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId
0,00025_00000_00000.jpg,48,43,6,5,43,38,25
1,00025_00000_00001.jpg,49,46,5,6,44,41,25
2,00025_00000_00002.jpg,49,46,5,6,44,41,25
3,00025_00000_00003.jpg,51,47,6,6,46,42,25
4,00025_00000_00004.jpg,48,46,5,5,43,41,25
...,...,...,...,...,...,...,...,...
39204,00001_00073_00025.jpg,74,77,7,7,68,71,1
39205,00001_00073_00026.jpg,76,80,7,7,70,72,1
39206,00001_00073_00027.jpg,79,83,7,8,72,76,1
39207,00001_00073_00028.jpg,82,85,8,8,75,78,1


In [ ]:
from sklearn.model_selection import train_test_split

y = train_csv['ClassId']
train_data, remaining_data = train_test_split(
    train_csv,
    test_size=0.3,
    random_state=43,
    shuffle=True,
    stratify=y
)

test_data, val_data= train_test_split(
    remaining_data,
    test_size=0.33,  # 1/3 of remaining data goes to validation, 2/3 goes to test
    random_state=43,
    shuffle=True,
)

# Print the sizes of each set
print("Train set size:", len(train_data))
print("Validation set size:", len(val_data))
print("Test set size:", len(test_data))

Train set size: 27446
Validation set size: 3882
Test set size: 7881


In [ ]:
os.makedirs('YOLO/images/train', exist_ok=True)
os.makedirs('YOLO/images/valid', exist_ok=True)
os.makedirs('YOLO/images/test', exist_ok=True)

In [ ]:
def move(data, split='train'):
  for index, x in data.iterrows():
    shutil.move(f'YOLO/images/{x.Filename}', f'YOLO/images/{split}')

move(train_data, 'train')
move(val_data, 'valid')
move(test_data, 'test')

In [ ]:
def compare(v1, v2):
    if v1 > v2:
        vmax, vmin = v1, v2
        return vmax, vmin
    else:
        vmax, vmin = v2, v1
        return vmax, vmin
def convert_labels(z):
    x1 = z['Roi.X1']
    y1 = z['Roi.Y1']
    x2 = z['Roi.X2']
    y2 = z['Roi.Y2']
    size = [z['Height'],z['Width']]
    xmax, xmin = compare(x1, x2)
    ymax, ymin = compare(y1, y2)
    dw = 1./size[1]
    dh = 1./size[0]
    x = (xmin + xmax)/2.0
    y = (ymin + ymax)/2.0
    w = xmax - xmin
    h = ymax - ymin
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return x,y,w,h

In [ ]:
os.makedirs('YOLO/labels/train', exist_ok=True)
os.makedirs('YOLO/labels/valid', exist_ok=True)
os.makedirs('YOLO/labels/test', exist_ok=True)

def create_annot(data, split='train'):
  for index, x in data.iterrows():
      name = x['Filename'].replace('.jpg','.txt')
      labels = convert_labels(x)
      cls = x['ClassId']
      f = open(f'YOLO/labels/{split}/{name}','w+')
      f.write(f'{cls} {labels[0]} {labels[1]} {labels[2]} {labels[3]}')
      f.close()

create_annot(train_data, 'train')
create_annot(val_data, 'valid')
create_annot(test_data, 'test')

In [ ]:
os.makedirs('YOLO/data', exist_ok=True)


def create_pathfile(data, split='train'):
  f = open(f'YOLO/data/{split}.text','w+')
  for index, x in data.iterrows():
      name = x['Filename']
      f.write(f'YOLO/images/{split}/{name}\n')
  f.close()

create_pathfile(train_data, 'train')
create_pathfile(val_data, 'valid')
create_pathfile(test_data, 'test')

In [ ]:
classes = [
    'Speed limit (20km/h)',
    'Speed limit (30km/h)',
    'Speed limit (50km/h)',
    'Speed limit (60km/h)',
    'Speed limit (70km/h)',
    'Speed limit (80km/h)',
    'Speed limit (100km/h)',
    'Speed limit (120km/h)',
    'No passing',
    'No passing for vechiles over 3.5 metric tons',
    'No vehicles',
    'Vechiles over 3.5 metric tons prohibited',
    'Right-of-way at the next intersection',
    'General caution',
    'Dangerous curve to the left',
    'Dangerous curve to the right',
    'Double curve',
    'Bumpy road',
    'Slippery road',
    'Road narrows on the right',
    'Road work',
    'Traffic signals',
    'Pedestrians',
    'Children crossing',
    'Bicycles crossing',
    'Beware of ice/snow',
    'Wild animals crossing',
    'Turn right ahead',
    'Turn left ahead',
    'Ahead only',
    'Go straight or right',
    'Go straight or left',
    'Keep right',
    'Keep left',
    'Roundabout mandatory',
    'Priority road',
    'Yield',
    'Stop',
    'No entry',
    'End of speed limit (80km/h)',
    'End of all speed and passing limits',
    'End of no passing',
    'End of no passing by vechiles over 3.5 metric tons'
]

os.makedirs('YOLO/data', exist_ok=True)
f = open(f'YOLO/data/classes.names','w+')
for c in classes:
    f.write(f'{c}\n')
f.close()

# Train

In [ ]:
!pip install ultralytics --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 699.7/699.7 kB 8.6 MB/s eta 0:00:00


In [ ]:
import os
import ultralytics

from ultralytics import YOLO


# Load a model
model = YOLO("yolov8n.yaml")  # build a new model from scratch



# Use the model
results = model.train( data="/content/YOLO/path.yaml", epochs=20)  # train the model


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

train: Scanning YOLO/labels/train.cache... 27446 images, 0 backgrounds, 0 corrupt: 100%|██████████| 27446/27446 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning YOLO/labels/valid.cache... 3882 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3882/3882 [00:00<?, ?it/s]


Plotting labels to runs/detect/train6/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000213, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      20/20       2.7G     0.4238     0.2566      1.131          6        640: 100%|██████████| 1716/1716 [08:03<00:00,  3.55it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 122/122 [00:33<00:00,  3.66it/s]


                   all       3882       3882      0.984      0.986      0.993      0.943

20 epochs completed in 3.060 hours.
Optimizer stripped from runs/detect/train6/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train6/weights/best.pt, 6.2MB

Validating runs/detect/train6/weights/best.pt...
Ultralytics YOLOv8.0.232 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n summary (fused): 168 layers, 3014033 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 122/122 [00:40<00:00,  3.05it/s]


                   all       3882       3882      0.983      0.986      0.993      0.943
  Speed limit (20km/h)       3882         23      0.984          1      0.995      0.956
  Speed limit (30km/h)       3882        241      0.999          1      0.995      0.954
  Speed limit (50km/h)       3882        219          1      0.991      0.995      0.944
  Speed limit (60km/h)       3882        139          1      0.999      0.995      0.951
  Speed limit (70km/h)       3882        208      0.999          1      0.995      0.956
  Speed limit (80km/h)       3882        173      0.994      0.993      0.995      0.929
 Speed limit (100km/h)       3882         55      0.995          1      0.995      0.906
 Speed limit (120km/h)       3882        145          1      0.986      0.995      0.942
            No passing       3882        127      0.995          1      0.995      0.921
No passing for vechiles over 3.5 metric tons       3882        152      0.997          1      0.995      0.937

In [ ]:
os.makedirs('/content/drive/MyDrive/gtsrb_yolov8')


AttributeError: ignored

In [ ]:
import os
import shutil

def move_files(source_folder, destination_folder):
    # Check if the source folder exists
    if not os.path.exists(source_folder):
        print(f"Source folder '{source_folder}' does not exist.")
        return

    # Check if the destination folder exists, create it if not
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Get a list of all files in the source folder
    files = os.listdir(source_folder)

    # Move each file to the destination folder
    for file in files:
        source_path = os.path.join(source_folder, file)
        destination_path = os.path.join(destination_folder, file)
        shutil.move(source_path, destination_path)
        print(f"Moved '{file}' to '{destination_folder}'.")

# Example usage
source_folder = "/content/runs/detect/train6"
destination_folder = "/content/drive/MyDrive/gtsrb_yolov8"

move_files(source_folder, destination_folder)


Moved 'confusion_matrix.png' to '/content/drive/MyDrive/gtsrb_yolov8'.
Moved 'P_curve.png' to '/content/drive/MyDrive/gtsrb_yolov8'.
Moved 'PR_curve.png' to '/content/drive/MyDrive/gtsrb_yolov8'.
Moved 'val_batch1_pred.jpg' to '/content/drive/MyDrive/gtsrb_yolov8'.
Moved 'train_batch0.jpg' to '/content/drive/MyDrive/gtsrb_yolov8'.
Moved 'train_batch17160.jpg' to '/content/drive/MyDrive/gtsrb_yolov8'.
Moved 'val_batch2_pred.jpg' to '/content/drive/MyDrive/gtsrb_yolov8'.
Moved 'train_batch1.jpg' to '/content/drive/MyDrive/gtsrb_yolov8'.
Moved 'val_batch0_pred.jpg' to '/content/drive/MyDrive/gtsrb_yolov8'.
Moved 'weights' to '/content/drive/MyDrive/gtsrb_yolov8'.
Moved 'labels.jpg' to '/content/drive/MyDrive/gtsrb_yolov8'.
Moved 'results.png' to '/content/drive/MyDrive/gtsrb_yolov8'.
Moved 'labels_correlogram.jpg' to '/content/drive/MyDrive/gtsrb_yolov8'.
Moved 'args.yaml' to '/content/drive/MyDrive/gtsrb_yolov8'.
Moved 'confusion_matrix_normalized.png' to '/content/drive/MyDrive/gtsrb_yo

# Inference

In [ ]:
!pip install ultralytics gradio typing_extensions --quiet

In [ ]:
import os
import ultralytics

from ultralytics import YOLO

model = YOLO('/content/drive/MyDrive/gtsrb_yolov8/weights/best.pt')
prediction = model.predict('/content/bocchi.gif')


In [ ]:
import gradio

In [ ]:
import cv2

In [ ]:
import gradio as gr
from ultralytics import YOLO
import cv2
import numpy as np

# Load your YOLOv5 model from Ultralytics
model = YOLO('/content/drive/MyDrive/gtsrb_yolov8/weights/best.pt')

def save_video(frames, output_video_path, fps=30):
    # Get the height and width from the first frame
    height, width, _ = frames[0].shape

    # Define the codec and create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # Write each frame to the video file
    for frame in frames:
        out.write(frame)

    # Release the VideoWriter
    out.release()

def predict_video(input_video_path, output_video_path):
    # Open the video file
    results = model.predict(input_video_path, conf=0.5, vid_stride=5)

    frames = [im.plot() for im in results]

    save_video(frames, 'result.mp4')
    return 'result.mp4'

# Define Gradio interface for video
iface = gr.Interface(
    fn=predict_video,
    inputs=gr.Video(),
    outputs="playable_video",
    examples=[os.path.join(os.path.abspath(''), "video/video_sample.mp4")],
    cache_examples=True)

# Launch the Gradio interface for video
iface.launch()

/usr/local/lib/python3.10/dist-packages/gradio/utils.py:840: UserWarning: Expected 2 arguments for function <function predict_video at 0x7be1bf7d8940>, received 1.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/utils.py:844: UserWarning: Expected at least 2 arguments for function <function predict_video at 0x7be1bf7d8940>, received 1.
  warnings.warn(


Caching examples at: '/content/gradio_cached_examples/111'
Caching example 1/1


/usr/local/lib/python3.10/dist-packages/gradio/utils.py:840: UserWarning: Expected 2 arguments for function <function predict_video at 0x7be1bf7d8940>, received 1.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/utils.py:844: UserWarning: Expected at least 2 arguments for function <function predict_video at 0x7be1bf7d8940>, received 1.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/helpers.py:837: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")




WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/30) /tmp/gradio/5e957c117a92e7e59be3f3051baead2c958844e3/video_sample.mp4: 384x640 (no detections), 196.9ms
video 1/1 (2/30) /tmp/gradio/5e957c117a92e7e59be3f3051baead2c958844e3/video_sample.mp4: 384x640 (no detections), 139.9ms
video 1/1 (3/30) /tmp/gradio/5e957c117a92e7e59be3f3051baead2c958844e3/video_sample.mp4: 384x640 (no detections), 144.2ms
video 1/1 (4/30) /tmp/gradio/5e957c117a92e7e59be3f3051baead2c958844e3/video_sample.mp4: 384x6

/usr/local/lib/python3.10/dist-packages/gradio/components/video.py:278: UserWarning: Video does not have browser-compatible container or codec. Converting to mp4
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://74542628d3ca3d1326.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
